<a href="https://colab.research.google.com/github/anhpdd/ml-property-valuation-klang-valley/blob/main/notebooks/3_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q osmnx

# Core libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx

# Geospatial
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from shapely import wkt
from geopy.distance import great_circle

# Visualization
import folium
from matplotlib.colors import Normalize, to_hex, LinearSegmentedColormap
from branca.colormap import LinearColormap

# Utilities
import requests
from tqdm import tqdm
from typing import List, Dict, Any, Optional
from IPython.display import display

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Functions

In [ ]:
def extract_line_geometry(input_df: pd.DataFrame, name_column,id_column) -> Optional[gpd.GeoDataFrame]:
    """
    Fetches OSM data for Way IDs found in the input DataFrame,
    parses road details and geometry, and returns a single GeoDataFrame
    containing all valid road LineStrings, merging with input DataFrame attributes.

    Args:
        input_df (pd.DataFrame): A DataFrame expected to have a 'Way ID' column
                                  and any other attributes to carry over.

    Returns:
        Optional[gpd.GeoDataFrame]: A GeoDataFrame with road geometries and
                                     merged attributes, or None if no valid data.
    """


    all_geometries = []
    all_attributes = []

    for index, row in input_df.iterrows():
        way_id = str(row[id_column]) # Ensure way_id is a string

        url = f"{OSM_API_BASE_URL}/way/{way_id}/full"
        root = fetch_osm_data(url)

        if not root:
            print(f"Skipping Way ID '{way_id}' due to data fetch/parse error.")
            continue

        road_way_elem = root.find(f".//way[@id='{way_id}']")
        if not road_way_elem:
            print(f"Warning: Road Way ID '{way_id}' not found in fetched XML for {url}. Skipping.")
            continue

        # Node caching for the *current* XML response
        node_coords_cache = {} # {node_id: (lon, lat)}
        for node_elem in root.findall(".//node"):
            try:
                node_id = node_elem.get('id')
                lat = float(node_elem.get('lat'))
                lon = float(node_elem.get('lon'))
                node_coords_cache[node_id] = (lon, lat)
            except (TypeError, ValueError):
                pass # Skip invalid nodes

        # Extract road attributes (tags) from OSM
        osm_road_attrs = {'id': way_id}
        for tag in road_way_elem.findall('tag'):
            osm_road_attrs[tag.get('k')] = tag.get('v')

        # Combine attributes from the input DataFrame row with OSM attributes
        # The input DataFrame attributes will take precedence if keys overlap
        merged_attrs = row.to_dict() # Start with attributes from the input row
        merged_attrs.update(osm_road_attrs) # Add/overwrite with OSM-fetched attributes

        # Build coordinates for the LineString
        road_coords = []
        for nd_ref_elem in road_way_elem.findall('nd'):
            node_ref = nd_ref_elem.get('ref')
            coords = node_coords_cache.get(node_ref)
            if coords:
                road_coords.append(coords)

        # Create LineString if enough points exist
        if len(road_coords) >= 2:
            line_geometry = LineString(road_coords)
            all_geometries.append(line_geometry)
            all_attributes.append(merged_attrs)
        else:
            print(f"Warning: Insufficient coordinates for Way ID '{way_id}' to form a LineString. Skipping.")

    if not all_geometries:
        print("No valid LineString geometries could be created for any of the provided Way IDs.")
        return None

    # Create a single GeoDataFrame from all collected geometries and attributes
    # The 'crs' (Coordinate Reference System) specifies WGS84, common for GPS coordinates.
    gdf = gpd.GeoDataFrame(all_attributes, geometry=all_geometries, crs="EPSG:4326")
    gdf = gdf.drop_duplicates(subset= name_column)
    gdf = gdf[[name_column, 'id', 'geometry']]

    return gdf

In [ ]:
def visualize_gdf_on_folium_colored_with_legend(
    gdf: gpd.GeoDataFrame,
    columns_to_display: Optional[List[str]] = None,
    residential_price_column: str = 'RM per Land m2',
    top_percentile_highlight: float = 0.90
) -> Optional[folium.Map]:
    if gdf.empty or 'geometry' not in gdf.columns:
        print("GeoDataFrame is empty or missing 'geometry' column.")
        return None

    valid_lines_gdf = gdf[gdf.geometry.apply(lambda geom: isinstance(geom, LineString) and not geom.is_empty)].copy()
    if valid_lines_gdf.empty:
        print("No valid LineString geometries found.")
        return None

    # --- Map Centering ---
    try:
    # Always convert to 4326 for centering and plotting
      temp_gdf = valid_lines_gdf.to_crs("EPSG:4326")
      map_center_geom = temp_gdf.geometry.unary_union.centroid
      m = folium.Map(
          location=[map_center_geom.y, map_center_geom.x],
          zoom_start=12,
          tiles='CartoDB positron'  # <-- ADD THIS LINE
      )
    except Exception as e:
        print(f"Could not determine map center: {e}. Using default location.")
        m = folium.Map(
            location=[3.1390, 101.6869],
            zoom_start=10,
            tiles='CartoDB positron'
        )

    # --- Price Normalization and Colormap ---
    min_price_val, max_price_val, top_percentile_threshold, norm_normal_range = None, None, None, None
    colors = ['green', 'yellow', 'orange']
    normal_residential_cmap = LinearSegmentedColormap.from_list('GnYeOr', colors)
    top_percentile_color_hex = to_hex('red')

    if residential_price_column not in valid_lines_gdf.columns:
        print(f"Error: Price column '{residential_price_column}' not found.")
        return m # Return map without colored lines

    prices_numeric = pd.to_numeric(valid_lines_gdf[residential_price_column], errors='coerce').dropna()
    if len(prices_numeric) > 1:
        top_percentile_threshold = prices_numeric.quantile(top_percentile_highlight)
        normal_range_prices = prices_numeric[prices_numeric < top_percentile_threshold]
        if not normal_range_prices.empty:
            min_price_val = normal_range_prices.min()
            max_price_val = normal_range_prices.max()
            norm_normal_range = Normalize(vmin=min_price_val, vmax=max_price_val) if min_price_val != max_price_val else Normalize(vmin=min_price_val, vmax=min_price_val + 1)

    # --- FIX 1: Reproject data to the correct CRS for Folium ---
    valid_lines_gdf = valid_lines_gdf.to_crs("EPSG:4326")

    # --- Main Loop to Draw Lines ---
    for idx, row in valid_lines_gdf.iterrows():
        line = row['geometry']
        folium_coords = [(point[1], point[0]) for point in line.coords]

        line_color = 'grey' # Default color
        current_price = pd.to_numeric(row.get(residential_price_column), errors='coerce')

        if pd.notna(current_price):
            if top_percentile_threshold and current_price >= top_percentile_threshold:
                line_color = top_percentile_color_hex
            elif norm_normal_range:
                line_color = to_hex(normal_residential_cmap(norm_normal_range(current_price)))

        folium.PolyLine(locations=folium_coords, color=line_color, weight=4, opacity=0.8).add_to(m)

    # --- FIX 2: Complete legend generation code ---
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; width: auto; height: auto;
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white; opacity:0.9; padding: 10px;">
        <b>Legend</b> <br>
    """
    if top_percentile_threshold is not None:
        legend_html += f'<i style="background:{top_percentile_color_hex}; opacity:0.8; border: 1px solid #000; display: inline-block; width: 12px; height: 12px; margin-right: 5px;"></i> Top {100*(1-top_percentile_highlight):.0f}% Price (≥ {top_percentile_threshold:,.0f})<br>'
    if norm_normal_range and min_price_val is not None and max_price_val is not None:
        legend_html += f'<b>Price (Normal Range)</b><br>'
        gradient_html = '<div style="background-image: linear-gradient(to right, green, yellow, orange); height: 20px; width: 100%; border: 1px solid #ccc;"></div>'
        legend_html += f'<div style="display: flex; justify-content: space-between;"><span>Low: {min_price_val:,.0f}</span><span>High: {max_price_val:,.0f}</span></div>'
        legend_html += gradient_html
    else:
        legend_html += '<i style="background:grey; opacity:0.8; border: 1px solid #000; display: inline-block; width: 12px; height: 12px; margin-right: 5px;"></i> Residential (Price N/A or uniform)<br>'
    legend_html += "</div>"
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

# Import data

In [ ]:
# Import property dataset
df_raw=pd.read_excel('/content/drive/MyDrive/Colab/Capstone 1/tprop_df_validated.xlsx')

# Inspect dataset
#df_raw.head()

In [ ]:
# Inspect datatypes
#df_raw.info()

In [ ]:
df = df_raw.copy()

# Convert the date column
df['date'] = pd.to_datetime(df['date'])

# Fix property size
df['property_m2'] = df['property_m2'] / 100
df['land_m2'] = df['land_m2'] / 100

# Binary tenure feature
df['freehold'] = (df['tenure'] == 'Freehold').astype(int)
df['price_m2'] = round(df['transaction_price'] / df['property_m2'], 2)

# Standardize strings
string_cols = df.select_dtypes(include=['object']).columns
string_cols = string_cols.drop('geometry', errors='ignore')
for col in string_cols:
    df[col] = df[col].str.strip().str.lower()


# Convert string geometries to actual Shapely objects
df['geometry'] = df['geometry'].apply(wkt.loads)

# Drop unnecessary columns
cols_to_drop = ['tenure']
df_clean = df.drop(columns=cols_to_drop, errors='ignore')

# Verify the new dtypes
#print(df_clean.info())
#df_clean.head()

# Visualize

In [ ]:
# Create the GeoDataFrame and set the CRS
gdf = gpd.GeoDataFrame(df_clean, geometry='geometry').set_crs(epsg=4326, inplace=True)

# Configure the map
#map_viz = visualize_gdf_on_folium_colored_with_legend(
#    gdf,
#    residential_price_column='price_m2'
#)

# Display the map
#map_viz

# Amnity ID Extract

In [ ]:
# Import amenity data
amenity_df = pd.read_excel('/content/drive/MyDrive/Colab/Capstone 1/all_pois_gdf.xlsx').drop(columns='Unnamed: 0').rename(columns = {'Station ID': 'station_id'})

# Set up geometry
amenity_df['geometry'] = amenity_df['geometry'].apply(wkt.loads)

# Convert it to GeoDataFrame and set crs to 4326
amenity_gdf = gpd.GeoDataFrame(amenity_df, geometry='geometry').set_crs(epsg=4326, inplace=True)

#Inspect df
#amenity_gdf.head()

In [ ]:
# Import ridership data
ridership = pd.read_excel('/content/drive/MyDrive/Colab/Capstone 1/ridership_data.xlsx')

# Set up date
ridership['date'] = pd.to_datetime(ridership['date'])

# Clean station ID column
ridership['station_id'] = ridership['station_name'].str.split(':').str[0]

# Inspect dataset
#ridership.head()

In [ ]:
# Inspect dataset datatypes
#ridership.info()

# Count

In [ ]:
def count_amenities(
    property_gdf: gpd.GeoDataFrame,
    amenity_gdf: gpd.GeoDataFrame,
    radius_km: float,
    amenity_type_column: str = 'feature_type',
    property_id_col: str = None,
    amenity_id_col: str = None,
    train_type_name: str = 'train'
) -> gpd.GeoDataFrame:
    """
    Buffer amenities and find which properties they affect - with auto-detection!
    """
    result_gdf = property_gdf.copy()

    # Auto-detect property ID column if not specified
    if property_id_col is None:
        # Look for common ID column names
        possible_names = ['id', 'ID', 'property_id', 'prop_id', 'index']
        for name in possible_names:
            if name in property_gdf.columns:
                property_id_col = name
                break

        if property_id_col is None:
            # Use the index
            print("No ID column found, using index as property ID")
            property_gdf = property_gdf.reset_index()
            property_id_col = 'index'
            result_gdf = property_gdf.copy()

    print(f"Using property ID column: '{property_id_col}'")

    # Auto-detect amenity ID column if not specified
    if amenity_id_col is None:
        possible_names = ['osm_id', 'id', 'ID', 'amenity_id']
        for name in possible_names:
            if name in amenity_gdf.columns:
                amenity_id_col = name
                break

        if amenity_id_col is None:
            # Create a temporary ID column
            print("No amenity ID column found, creating temporary IDs")
            amenity_gdf = amenity_gdf.copy()
            amenity_gdf['temp_amenity_id'] = range(len(amenity_gdf))
            amenity_id_col = 'temp_amenity_id'

    print(f"Using amenity ID column: '{amenity_id_col}'")

    # Match CRS
    if property_gdf.crs != amenity_gdf.crs:
        print("Converting CRS to match...")
        amenity_gdf = amenity_gdf.to_crs(property_gdf.crs)

    # Buffer AMENITIES with progress bar
    print(f"\nCreating {radius_km}km buffers around {len(amenity_gdf)} amenities...")
    radius_degrees = radius_km / 111.0
    amenity_buffered = amenity_gdf.copy()

    tqdm.pandas(desc="Buffering amenities")
    amenity_buffered['geometry'] = amenity_buffered['geometry'].progress_apply(
        lambda geom: geom.buffer(radius_degrees)
    )

    # Spatial join: which properties does each amenity affect?
    print("\nFinding affected properties (spatial join)...")
    affected = gpd.sjoin(
        amenity_buffered[[amenity_id_col, amenity_type_column, 'geometry']],
        property_gdf[[property_id_col, 'geometry']],
        how='inner',
        predicate='intersects'
    )

    print(f"✓ Found {len(affected)} amenity-property pairs")

    # Initialize result columns
    unique_types = amenity_gdf[amenity_type_column].dropna().unique()
    print(f"\nAmenity types found: {', '.join(str(t) for t in unique_types)}")

    for amenity_type in unique_types:
        type_name = str(amenity_type).lower().replace(' ', '_')
        result_gdf[f"{type_name}_count"] = 0

    result_gdf["total_amenity_count"] = 0
    result_gdf["train_ids"] = ""

    if len(affected) > 0:
        # Count amenities by type for each property
        print("\nCounting amenities by type...")
        counts = affected.groupby([property_id_col, amenity_type_column]).size().unstack(fill_value=0)

        # Map counts back to result with progress bar
        print("Mapping counts to properties...")
        for amenity_type in tqdm(counts.columns, desc="Processing types"):
            type_name = str(amenity_type).lower().replace(' ', '_')
            col_name = f"{type_name}_count"
            if col_name in result_gdf.columns:
                result_gdf[col_name] = result_gdf[property_id_col].map(
                    counts[amenity_type].to_dict()
                ).fillna(0).astype(int)

        # Total count per property
        print("Calculating total counts...")
        total = affected.groupby(property_id_col).size()
        result_gdf['total_amenity_count'] = result_gdf[property_id_col].map(
            total.to_dict()
        ).fillna(0).astype(int)

        # Collect train IDs
        print("Collecting train IDs...")
        train_affected = affected[affected[amenity_type_column].str.lower() == train_type_name.lower()]
        if len(train_affected) > 0:
            train_ids = train_affected.groupby(property_id_col)[amenity_id_col].apply(
                lambda x: ', '.join(x.astype(str).unique())
            )
            result_gdf['train_ids'] = result_gdf[property_id_col].map(
                train_ids.to_dict()
            ).fillna("")

    print(f"\n{'='*50}")
    print(f"✓ Done!")
    print(f"{'='*50}")
    print(f"Properties affected by at least one amenity: {(result_gdf['total_amenity_count'] > 0).sum()}/{len(result_gdf)}")
    print(f"Properties with trains nearby: {(result_gdf['train_ids'] != '').sum()}/{len(result_gdf)}")

    return result_gdf

In [ ]:
# Count amenity
result = count_amenities(
    property_gdf=gdf,
    amenity_gdf=amenity_gdf,
    radius_km=1.0,
    train_type_name='rail station'
)

# Check what train-related types exist in your data
print("Available amenity types:")
print(amenity_gdf['feature_type'].value_counts())

In [ ]:
def visualize_amenity_radius(
    property_gdf: gpd.GeoDataFrame,
    amenity_gdf: gpd.GeoDataFrame,
    radius_km: float,
    property_id: str = None,
    map_tiles: str = 'CartoDB positron',
    show_ridership: bool = True,
    ridership_column: str = 'total_ridership_within_1km',
    property_id_col: str = 'id'
) -> folium.Map:
    """
    Visualizes properties and amenities with optional ridership data.
    Args:
        property_gdf: GeoDataFrame with properties
        amenity_gdf: GeoDataFrame with amenities
        radius_km: Search radius in km
        property_id: Specific property to highlight
        map_tiles: Folium tile style
        show_ridership: Whether to visualize ridership data
        ridership_column: Column name for property ridership totals
        property_id_col: Column name for property ID (default 'id')
    """
    # Setup and CRS Alignment
    if property_gdf.empty or 'geometry' not in property_gdf.columns:
        print("Property GeoDataFrame is empty or missing geometry.")
        return None

    if amenity_gdf.crs != property_gdf.crs:
        amenity_gdf = amenity_gdf.to_crs(property_gdf.crs)

    # Check if property_id_col exists, if not use index
    if property_id_col not in property_gdf.columns:
        print(f"Warning: '{property_id_col}' column not found. Using index as ID.")
        property_gdf = property_gdf.copy()
        property_gdf[property_id_col] = property_gdf.index.astype(str)

    # Create Base Map
    try:
        center_point = property_gdf.to_crs("EPSG:4326").union_all().centroid
    except AttributeError:
        # Fallback for older geopandas versions
        center_point = property_gdf.to_crs("EPSG:4326").unary_union.centroid
    except Exception:
        # Fallback to default location
        center_point = None

    if center_point:
        m = folium.Map(location=[center_point.y, center_point.x], zoom_start=13, tiles=map_tiles)
    else:
        m = folium.Map(location=[3.1390, 101.6869], zoom_start=12, tiles=map_tiles)

    # Define Colors and Styles
    amenity_color_map = {
        'rail station': '#e74c3c',  # Red for rail
        'mall': '#9b59b6',          # Purple
        'school': '#f39c12',        # Orange
        'park': '#2ecc71',          # Green
        'lake': '#3498db',          # Blue
        'river': '#34495e',         # Dark grey
    }

    # Create ridership colormap for rail stations
    ridership_colormap = None
    if show_ridership and 'total_ridership' in amenity_gdf.columns:
        rail_stations = amenity_gdf[amenity_gdf['feature_type'] == 'rail station']
        if not rail_stations.empty and rail_stations['total_ridership'].max() > 0:
            ridership_colormap = LinearColormap(
                colors=['yellow', 'orange', 'red', 'darkred'],
                vmin=rail_stations['total_ridership'].min(),
                vmax=rail_stations['total_ridership'].max(),
                caption='Station Ridership (Total Incoming + Outgoing)'
            )

    # Find Highlighted Property
    highlight_property = None
    if property_id:
        matches = property_gdf[property_gdf[property_id_col] == property_id]
        if not matches.empty:
            highlight_property = matches.iloc[0]
    if highlight_property is None and not property_gdf.empty:
        highlight_property = property_gdf.iloc[0]

    # Create Layers
    property_layer = folium.FeatureGroup(name='Properties', show=True)
    amenity_layer = folium.FeatureGroup(name='Amenities', show=True)

    # Add Properties
    for idx, row in property_gdf.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue

        is_highlighted = highlight_property is not None and row[property_id_col] == highlight_property[property_id_col]

        # Style based on ridership if available
        if show_ridership and ridership_column in row and not pd.isna(row[ridership_column]):
            ridership = row[ridership_column]
            # Color intensity based on ridership
            if ridership > 50_000_000:
                color = '#8B0000'  # Dark red
                weight = 6
            elif ridership > 20_000_000:
                color = '#FF0000'  # Red
                weight = 5
            elif ridership > 5_000_000:
                color = '#FF6347'  # Tomato
                weight = 4
            elif ridership > 0:
                color = '#FFA500'  # Orange
                weight = 3
            else:
                color = '#6c757d'  # Grey
                weight = 3
        else:
            color = 'red' if is_highlighted else '#6c757d'
            weight = 5 if is_highlighted else 3

        # Convert to lat/lon
        g = gpd.GeoSeries.from_wkt([geom.wkt], crs=property_gdf.crs).to_crs("EPSG:4326").iloc[0]

        # Create popup with property info
        popup_html = f"<b>{row.get('road_name', 'Property')}</b><br>"
        popup_html += f"ID: {row.get(property_id_col, 'N/A')}<br>"
        if show_ridership and ridership_column in row and not pd.isna(row[ridership_column]):
            ridership_val = row[ridership_column]
            popup_html += f"<b>Total Ridership (1km): {ridership_val:,.0f}</b><br>"
        if 'rail_station_count' in row:
            popup_html += f"Rail Stations Nearby: {row['rail_station_count']}<br>"

        # Add distance columns if present
        for col in row.index:
            if col.startswith('dist_to_'):
                amenity_name = col.replace('dist_to_', '').replace('_', ' ').title()
                dist_val = row[col]
                if not pd.isna(dist_val):
                    popup_html += f"{amenity_name}: {dist_val:.2f} km<br>"

        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in g.coords],
            color=color,
            weight=weight,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=row.get('road_name', 'Property')
        ).add_to(property_layer)

        # Add radius circle
        if is_highlighted:
            center_lat, center_lon = g.centroid.y, g.centroid.x
            folium.Circle(
                location=[center_lat, center_lon],
                radius=radius_km * 1000,
                color='red',
                fill=True,
                fill_opacity=0.1,
                tooltip=f"{radius_km} km radius"
            ).add_to(property_layer)

    # Add Amenities
    for idx, row in amenity_gdf.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue

        feature_type = str(row.get('feature_type', 'unknown')).lower()

        # Determine color
        if feature_type == 'rail station' and show_ridership and ridership_colormap and 'total_ridership' in row:
            ridership = row.get('total_ridership', 0)
            if ridership > 0:
                color = ridership_colormap(ridership)
            else:
                color = amenity_color_map.get(feature_type, 'gray')
        else:
            color = amenity_color_map.get(feature_type, 'gray')

        # Create popup
        popup_html = f"<b>{row.get('name', 'Unnamed')}</b><br>"
        popup_html += f"Type: {feature_type.title()}<br>"
        if 'station_id' in row:
            popup_html += f"Station ID: {row['station_id']}<br>"
        if show_ridership and 'total_ridership' in row and row['total_ridership'] > 0:
            popup_html += f"<b>Total Ridership: {row['total_ridership']:,.0f}</b><br>"

        # Convert to lat/lon
        g = gpd.GeoSeries.from_wkt([geom.wkt], crs=amenity_gdf.crs).to_crs("EPSG:4326").iloc[0]

        if g.geom_type == 'Point':
            # Larger markers for rail stations
            radius = 8 if feature_type == 'rail station' else 5
            folium.CircleMarker(
                location=[g.y, g.x],
                popup=folium.Popup(popup_html, max_width=250),
                tooltip=row.get('name', feature_type.title()),
                radius=radius,
                color=color,
                fill=True,
                fill_opacity=0.8,
                weight=2
            ).add_to(amenity_layer)
        elif g.geom_type == 'LineString':
            folium.PolyLine(
                locations=[(lat, lon) for lon, lat in g.coords],
                popup=folium.Popup(popup_html, max_width=250),
                color=color,
                weight=3
            ).add_to(amenity_layer)
        elif g.geom_type in ['Polygon', 'MultiPolygon']:
            folium.GeoJson(
                g,
                popup=folium.Popup(popup_html, max_width=250),
                style_function=lambda x, color=color: {
                    'fillColor': color,
                    'color': color,
                    'weight': 2,
                    'fillOpacity': 0.4,
                }
            ).add_to(amenity_layer)

    # Add Layers to Map
    property_layer.add_to(m)
    amenity_layer.add_to(m)

    # Create Legend
    legend_html = f'''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 250px;
                border:2px solid grey; z-index:9999; font-size:13px;
                background-color:white; padding: 10px; opacity: 0.9;">
        <div style="font-weight: bold; margin-bottom: 8px; font-size: 15px;">Legend</div>
        <div><i style="background:red; width:15px; height:3px; display:inline-block; margin-right:5px;"></i>Highlighted Property</div>
        <div><i style="border: 2px solid red; border-radius:50%; width:12px; height:12px; display:inline-block; margin-right:5px;"></i>Search Radius ({radius_km} km)</div>
        <hr style="margin: 8px 0;">
        <div style="font-weight: bold; margin-bottom: 5px;">Amenity Types</div>
    '''

    for f_type, f_color in amenity_color_map.items():
        legend_html += f'<div><i style="background:{f_color}; width:12px; height:12px; border-radius:50%; display:inline-block; margin-right:5px;"></i>{f_type.title()}</div>'

    if show_ridership and ridership_colormap:
        legend_html += '<hr style="margin: 8px 0;">'
        legend_html += '<div style="font-weight: bold; margin-bottom: 5px;">Rail Station Ridership</div>'
        legend_html += '<div style="font-size: 11px;">Darker = Higher Ridership</div>'

    legend_html += '</div>'

    m.get_root().html.add_child(folium.Element(legend_html))

    # Add ridership colormap if available
    if ridership_colormap:
        ridership_colormap.add_to(m)

    # Add Controls
    folium.LayerControl(collapsed=False).add_to(m)

    return m

In [ ]:
search_radius_km = 1
map = visualize_amenity_radius(
        result,
        amenity_gdf,
        search_radius_km,
        property_id_col='records_id'
    )
map

In [ ]:
def add_ridership_by_station_id_with_date(
    property_gdf,
    amenity_gdf,
    ridership_df,
    radius_km=1.0,
    property_id_col='index',
    property_date_col='date',
    ridership_date_col='date',
    amenity_id_col='osm_id'
):
    """
    Join ridership to properties matching by station_id AND date.
    Returns separate columns for incoming and outgoing ridership.
    """

    # Prepare ridership data
    print("Preparing ridership data...")
    ridership_clean = ridership_df[ridership_df['station_id'] != 'A0'].copy()
    ridership_clean[ridership_date_col] = pd.to_datetime(ridership_clean[ridership_date_col])

    station_ridership = ridership_clean.groupby(['station_id', ridership_date_col]).agg({
        'incoming': 'sum',
        'outgoing': 'sum'
    }).reset_index()

    # Keep incoming and outgoing separate
    station_ridership['total_ridership'] = (
        station_ridership['incoming'] + station_ridership['outgoing']
    )

    print(f"✓ Prepared ridership for {len(station_ridership['station_id'].unique())} stations")
    print(f"  Date range: {station_ridership[ridership_date_col].min()} to {station_ridership[ridership_date_col].max()}")

    # Join ridership with rail stations
    print("\nJoining ridership with station locations...")
    rail_stations = amenity_gdf[amenity_gdf['feature_type'] == 'rail station'].copy()

    # Rename date column to avoid conflicts in spatial join
    station_ridership = station_ridership.rename(columns={ridership_date_col: 'ridership_date'})

    # Merge with ALL ridership columns (incoming, outgoing, total)
    stations_with_ridership = rail_stations.merge(
        station_ridership[['station_id', 'ridership_date', 'incoming', 'outgoing', 'total_ridership']],
        on='station_id',
        how='left'
    )

    print(f"✓ Created {len(stations_with_ridership):,} station-date combinations")

    # Project and buffer
    print(f"Buffering stations ({radius_km}km)...")
    stations_projected = stations_with_ridership.to_crs('EPSG:32648')
    property_projected = property_gdf.copy()
    property_projected = property_projected.to_crs('EPSG:32648')

    # Rename property date column to avoid conflicts
    property_projected = property_projected.rename(columns={property_date_col: 'property_date'})

    radius_meters = radius_km * 1000
    stations_buffered = stations_projected.copy()
    stations_buffered['geometry'] = stations_buffered.geometry.buffer(radius_meters)

    # Spatial join
    print("Finding affected properties...")
    affected = gpd.sjoin(
        stations_buffered[[amenity_id_col, 'station_id', 'ridership_date', 'incoming', 'outgoing', 'total_ridership', 'geometry']],
        property_projected[[property_id_col, 'property_date', 'geometry']],
        how='inner',
        predicate='intersects'
    )

    print(f"✓ Found {len(affected):,} station-property-date combinations")

    # Filter to matching dates
    print(f"Matching dates between properties and ridership...")

    # Ensure both are datetime
    affected['property_date'] = pd.to_datetime(affected['property_date'])
    affected['ridership_date'] = pd.to_datetime(affected['ridership_date'])

    # Keep only matching dates
    affected_matched = affected[affected['property_date'] == affected['ridership_date']].copy()

    print(f"✓ Matched {len(affected_matched):,} station-property pairs with same dates")

    if len(affected_matched) == 0:
        print("\n⚠ Warning: No date matches found!")
        print(f"\nProperty dates sample:")
        print(property_gdf[property_date_col].dt.date.unique()[:10])
        print(f"\nRidership dates sample:")
        print(ridership_clean[ridership_date_col].dt.date.unique()[:10])

        # Check overlap
        prop_dates = set(pd.to_datetime(property_gdf[property_date_col]).dt.date)
        rider_dates = set(pd.to_datetime(ridership_clean[ridership_date_col]).dt.date)
        overlap = prop_dates.intersection(rider_dates)
        print(f"\nOverlapping dates: {len(overlap)}")
        if len(overlap) > 0:
            print(f"Examples: {list(overlap)[:5]}")

        return None

    # Step 6: Sum ridership per property (SEPARATE incoming and outgoing)
    print("Summing ridership for each property...")
    ridership_sum = affected_matched.groupby(property_id_col).agg({
        'incoming': 'sum',
        'outgoing': 'sum',
        'total_ridership': 'sum'
    }).reset_index()

    # Rename columns to be descriptive
    ridership_sum.columns = [
        property_id_col,
        'incoming_ridership_within_1km',
        'outgoing_ridership_within_1km',
        'total_ridership_within_1km'
    ]

    # Step 7: Join back to original properties
    result_gdf = property_gdf.copy()
    result_gdf = result_gdf.merge(ridership_sum, on=property_id_col, how='left')

    # Fill NaN with 0
    result_gdf['incoming_ridership_within_1km'] = result_gdf['incoming_ridership_within_1km'].fillna(0)
    result_gdf['outgoing_ridership_within_1km'] = result_gdf['outgoing_ridership_within_1km'].fillna(0)
    result_gdf['total_ridership_within_1km'] = result_gdf['total_ridership_within_1km'].fillna(0)

    # Summary
    print(f"\n{'='*60}")
    print("✓ COMPLETE!")
    print(f"{'='*60}")
    has_ridership = (result_gdf['total_ridership_within_1km'] > 0).sum()
    print(f"Properties with ridership: {has_ridership:,}/{len(result_gdf):,} ({has_ridership/len(result_gdf)*100:.1f}%)")
    print(f"\nRidership Statistics:")
    print(f"  Incoming - Avg: {result_gdf['incoming_ridership_within_1km'].mean():,.0f}, Max: {result_gdf['incoming_ridership_within_1km'].max():,.0f}")
    print(f"  Outgoing - Avg: {result_gdf['outgoing_ridership_within_1km'].mean():,.0f}, Max: {result_gdf['outgoing_ridership_within_1km'].max():,.0f}")
    print(f"  Total    - Avg: {result_gdf['total_ridership_within_1km'].mean():,.0f}, Max: {result_gdf['total_ridership_within_1km'].max():,.0f}")
    print(f"{'='*60}")

    return result_gdf

In [ ]:
# Calculate ridership
result_with_ridership = add_ridership_by_station_id_with_date(
    property_gdf=result,
    amenity_gdf=amenity_gdf,
    ridership_df=ridership,
    radius_km=1.0,
    property_date_col='date',
    ridership_date_col='date'
)

## Ridership check

In [ ]:
# Define the IDs to exclude
## Excluding lines from Shah Alam and most in PYL and 1 KJ37
exclude_ids = [
    10211729882, 7932664086, 10211729880, 10211729878, 10211729876,
    10211729874, 10211729872, 10211729870, 7725136738, 10211729860,
    10211729858, 10211729856, 10211729854, 10211729852, 10132470682,
    10211729850, 7724226582, 10211729846, 10211729845, 10211729843,
    10211729841, 10211729839, 9755011838, 9755011836, 9755011840,
    9833287690, 9833287692, 9833371330, 9849953874, 9849953872,
    7723739344, 7723739292, 10605496579, 10605496582, 10605496584,
    10605496586, 10605496587, 10605496590,10605496592,10605496594,
    10605496596, 10605496598, 10605496600, 10605496604,5702102258, 10605504105,
    5702102312, 5702102333,10605504107, 10591412710, 10605504109, 10605504111, 7077855554, 7723703874
]



# Convert to string for comparison since train_ids appears to be strings
exclude_ids_str = [str(id) for id in exclude_ids]

# Filter using a custom function to check if ANY excluded ID is in train_ids
def contains_excluded_id(train_ids_str, exclude_list):
    """Check if train_ids string contains any excluded IDs"""
    if train_ids_str == "":
        return False
    # Split the train_ids by comma
    ids_in_cell = [id.strip() for id in train_ids_str.split(',')]
    # Check if any ID is in exclude list
    return any(id in exclude_list for id in ids_in_cell)

# Apply filter
mask = (
    (result_with_ridership['train_ids'] != "") &
    (result_with_ridership['total_ridership_within_1km'] == 0) &
    (~result_with_ridership['train_ids'].apply(lambda x: contains_excluded_id(x, exclude_ids_str)))
)

filtered_result = result_with_ridership[mask][['train_ids']].value_counts()
print(filtered_result)

#Dist

In [ ]:
def download_road_graph(place_osm_ids: list, network_type: str = 'drive') -> nx.MultiDiGraph:
    """
    Downloads the road network for a specified list of OSM area IDs and merges them.
    """
    print(f"Downloading road networks for {len(place_osm_ids)} areas...")
    try:
        # Download boundary polygons INDIVIDUALLY for each OSM ID
        boundaries_list = []
        for osm_id in place_osm_ids:
            query_string = f"R{osm_id}"
            boundary_gdf = ox.geocode_to_gdf(query_string, by_osmid=True)
            boundaries_list.append(boundary_gdf)

        # Combine all boundaries into one GeoDataFrame
        boundaries_gdf = pd.concat(boundaries_list, ignore_index=True)
        print(f"✅ Retrieved {len(boundaries_gdf)} boundary polygons")

        # Download the graph for each polygon individually
        graphs = []
        for idx, row in boundaries_gdf.iterrows():
            polygon = row['geometry']
            area_name = row['display_name'].split(',')[0]
            print(f" -> Downloading '{network_type}' network for {area_name}...")
            G_area = ox.graph_from_polygon(polygon, network_type=network_type)
            graphs.append(G_area)

        # Merge all the individual graphs into one
        print("\nComposing individual graphs into a single network...")
        G_combined = nx.compose_all(graphs)
        print(f"✅ Successfully downloaded and combined {len(graphs)} graphs.")

        return G_combined

    except Exception as e:
        print(f"An error occurred during the graph download process: {e}")
        return None

def get_representative_point(geom):
    """
    Helper function to get a representative (latitude, longitude) point from various geometry types.
    For lines and polygons, it returns the centroid.
    """
    if geom is None or geom.is_empty:
        return None, None
    if isinstance(geom, Point):
        return geom.y, geom.x
    # For lines and polygons, the centroid is a good representative point
    centroid = geom.centroid
    return centroid.y, centroid.x

def calculate_travel_distances_fastest(
    property_gdf: gpd.GeoDataFrame,
    amenity_gdf: gpd.GeoDataFrame,
    G: nx.MultiDiGraph,
    max_distance_km: float = 3000.0
) -> gpd.GeoDataFrame:
    """
    FASTEST approach: Calculate from each property to nearest amenities
    """
    print("Ultra-optimized calculation started...")

    # CRS alignment
    graph_crs = G.graph['crs']
    if property_gdf.crs != graph_crs:
        property_gdf = property_gdf.to_crs(graph_crs)
    if amenity_gdf.crs != graph_crs:
        amenity_gdf = amenity_gdf.to_crs(graph_crs)

    result_gdf = property_gdf.copy()
    cutoff_meters = max_distance_km * 1000

    # Map to nodes
    print("Mapping to network...")
    prop_lats = result_gdf.geometry.apply(lambda g: get_representative_point(g)[0])
    prop_lons = result_gdf.geometry.apply(lambda g: get_representative_point(g)[1])
    property_nodes = pd.Series(
        ox.nearest_nodes(G, prop_lons.to_list(), prop_lats.to_list()),
        index=result_gdf.index
    )

    amenity_lats = amenity_gdf.geometry.apply(lambda g: get_representative_point(g)[0])
    amenity_lons = amenity_gdf.geometry.apply(lambda g: get_representative_point(g)[1])
    amenity_nodes = pd.Series(
        ox.nearest_nodes(G, amenity_lons.to_list(), amenity_lats.to_list()),
        index=amenity_gdf.index
    )

    # Create amenity type lookup
    amenity_type_nodes = {}
    for amenity_type in amenity_gdf['feature_type'].dropna().unique():
        amenity_indices = amenity_gdf[amenity_gdf['feature_type'] == amenity_type].index
        amenity_type_nodes[amenity_type] = set(amenity_nodes.loc[amenity_indices])

        # Initialize column
        col_name = f"dist_to_{str(amenity_type).lower().replace(' ', '_')}"
        result_gdf[col_name] = np.nan

    # Process each property
    print("Calculating distances from each property...")
    for idx, prop_node in tqdm(property_nodes.items(), total=len(property_nodes)):
        try:
            # Calculate distances from this property to nearby nodes
            distances = nx.single_source_dijkstra_path_length(
                G, prop_node, cutoff=cutoff_meters, weight='length'
            )

            # For each amenity type, find minimum distance
            for amenity_type, target_nodes in amenity_type_nodes.items():
                col_name = f"dist_to_{str(amenity_type).lower().replace(' ', '_')}"

                # Find minimum distance to any amenity of this type
                min_dist = min(
                    (distances[node] for node in target_nodes if node in distances),
                    default=np.nan
                )

                result_gdf.at[idx, col_name] = min_dist / 1000  # Convert to km

        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue

    print("Calculation complete.")
    return result_gdf

In [ ]:
import os
# Define the OSM IDs for the combined road network area.
DISTRICT_OSM_IDS = [2939672, # Kuala Lumpur
                     4443881, #Putrajaya
                     12438352, #GOMBAK
                     12438351, #HULU LANGAT
                     12391135, #KLANG
                     10714199, #HULU SELANGOR
                     10743362, #KUALA LANGAT
                     10714137, #KUALA SELANGOR
                     12391134, #PETALING
                     10714136, #SABAK BERNAM
                     10743315 #SEPANG
                     ]

# Define the save path
graph_filepath = '/content/drive/MyDrive/Colab/Capstone 1/klang_valley_road_network.graphml'

# Check if graph already exists, if not download and save it
if os.path.exists(graph_filepath):
    print("Loading existing road network graph...")
    road_network_graph = ox.load_graphml(graph_filepath)
    print("✅ Graph loaded from file!")
else:
    print("Downloading road network graph...")
    road_network_graph = download_road_graph(
        place_osm_ids=DISTRICT_OSM_IDS,
        network_type='drive'
    )

    # Save the graph
    print("Saving road network graph...")
    ox.save_graphml(road_network_graph, filepath=graph_filepath)
    print(f"✅ Graph saved to {graph_filepath}")


In [ ]:
# Define the walking save path
walk_graph_filepath = '/content/drive/MyDrive/Colab/Capstone 1/klang_valley_walking_network.graphml'

# Check if graph already exists, if not download and save it
if os.path.exists(walk_graph_filepath):
    print("Loading existing walking network graph...")
    roadwalk_network_graph = ox.load_graphml(walk_graph_filepath)
    print("✅ Graph loaded from file!")
else:
    print("Downloading walking network graph...")
    roadwalk_network_graph = download_road_graph(
        place_osm_ids=DISTRICT_OSM_IDS,
        network_type='walk'
    )

    # Save the graph
    print("Saving walking network graph...")
    ox.save_graphml(roadwalk_network_graph, filepath=walk_graph_filepath)
    print(f"✅ Graph saved to {walk_graph_filepath}")


In [ ]:
# Remove duplicate properties by records_id (not geometry)
property_gdf_unique = result_with_ridership.drop_duplicates(subset=['way_id'])

# Calculate WALKING distances
df_walk_result = calculate_travel_distances_fastest(
    property_gdf=property_gdf_unique,
    amenity_gdf=amenity_gdf,
    G=roadwalk_network_graph,
    max_distance_km=3000.0
)

# Rename walking distance columns
distance_cols = [col for col in df_walk_result.columns if col.startswith('dist_to_')]
rename_dict = {col: f'walk_{col}' for col in distance_cols}
df_walk_result.rename(columns=rename_dict, inplace=True)

# Calculate DRIVING distances
df_drive_result = calculate_travel_distances_fastest(
    property_gdf=property_gdf_unique,
    amenity_gdf=amenity_gdf,
    G=road_network_graph,
    max_distance_km=3000.0
)

# Merge walking + driving on records_id
walk_cols = ['way_id'] + [col for col in df_walk_result.columns if col.startswith('walk_')]
df_combined = df_walk_result[walk_cols].merge(
    df_drive_result[['way_id'] + [col for col in df_drive_result.columns if col.startswith('dist_to_')]],
    on='way_id',
    how='outer'  # Use outer to keep all properties
)

# Join back to original
optimized_distances_df = result_with_ridership.merge(
    df_combined,
    on='way_id',
    how='left'
)

In [ ]:
# Select distance columns
dist_cols = [col for col in optimized_distances_df.columns if 'dist_to_' in col]

# Check data quality
print(f"\n📊 Distance columns: {len(dist_cols)}")
print(f"❌ Missing values:\n{optimized_distances_df[dist_cols].isna().sum()}")

# Clean up dataframe
optimized_distances_df = (
    optimized_distances_df
    .filter(regex='^(?!.*_y$)')  # Remove _y columns
    .rename(columns=lambda x: x.replace('_x', ''))  # Rename _x suffixes
    .round({col: 4 for col in dist_cols})  # Round distances to 4 decimals
)

# Preview results
print(f"\n✅ Final shape: {optimized_distances_df.shape}")
optimized_distances_df.head()

In [ ]:
# Export dataset for further analysis
#optimized_distances_df.to_excel("/content/drive/MyDrive/Colab/Capstone 1/optimized_distances_df.xlsx")